In [1]:
from datetime import datetime

import matplotlib.pyplot as plt

from wasserstand.config import DATAFILE_TEMPLATE
from wasserstand.dataset import build_time_series, load_data_range
from wasserstand.models.high_level_predictor import HighLevelPredictor
from wasserstand.models.univariate import UnivariatePredictor
from wasserstand.models.mean import MeanPredictor

In [ ]:
training_data = build_time_series(
    load_data_range(datetime(2021, 10, 30), datetime(2021, 11, 1), DATAFILE_TEMPLATE)
)

testing_data = build_time_series(
    load_data_range(datetime(2021, 11, 1), datetime(2021, 11, 5), DATAFILE_TEMPLATE)
)

In [ ]:
plt.plot(training_data.time, training_data.sel(station="Innsbruck"))
plt.plot(testing_data.time, testing_data.sel(station="Innsbruck"))

In [ ]:
model = HighLevelPredictor(UnivariatePredictor(4, 1e-3, 1e-6))
model.fit(training_data)

In [ ]:
plt.plot(training_data.time, training_data.sel(station="Innsbruck"))
plt.plot(testing_data.time, testing_data.sel(station="Innsbruck"))

pred = model.predict_series(training_data)
plt.plot(pred.time, pred.sel(station="Innsbruck"))

pred = model.predict_series(testing_data)
plt.plot(pred.time, pred.sel(station="Innsbruck"))